# Exploratory Data Analysis (Group) on ACLR dataset

1. Our audience/stakeholder would be the doctor of a patient who has undergone ACLR surgery.
2. Our Problem Statment: Physicians need an efficient way to compare their patients progress with similar patients. Can we visualize patient data in a way that immediately provides a comparison of the individual patient to their peer group.
3. Important Variables (this is not an all emcompassing list):

These variables are important for finding the peer group:
*   vist_sex
*   age_group_dashboard_use
* graft_dashboard2


These variables are helpful for tracking the progress:


*   redcap_event_name
*   tss_dashboard
* koos_pain
* acl sh
* acl_th
* acl_ch
* acl_ext_mvic_90
* alc_ext_mvic_60
* acl_flex_mvic_60
* acl_ext_isok_60
* acl_flex_isok_60

There are potentially other variables which could be helpful for tracking patient progress, however we have not selected further important variables.




### Plan outlook:
Our current plan is to create a dynamic dashboard, in which the user (let it be a physician, surgeon, or a doctor) could select a group of data based on demographic variables, and then enter their current patient's stats/data to compare their patient's recovery status with the average/median recovery progress of **all the patients ** in that particular selected demographic group.

### Example Scenario:
If a doctor conducted a self-reported evaluation survey with their patient (i.e. KOOS), or any other form of strength evaluation based off a certain testing methodology. The doctor could enter that patient's survey results (or evaluation score) into the database, select the relevant demographic group that pertains to that particular patient, the dashboard would then compare the patient's score with the average/median score of all the patients in that demographic group. The dashboard would also provide a visual representation of the data, such as a bar graph or line chart, to help the doctor understand how their patient is doing compared to others in the same demographic group. This could help the doctor make more informed decisions about the patient's treatment plan and recovery process.

In [ ]:
!pip install -q pandas numpy matplotlib seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df= pd.read_csv('aclr.csv')
df.head()

## DATA CLEANING

Removing unnecessary columns (columns with zero data)

In [ ]:
df.columns[df.isnull().all()]# these are columns that have no data ( we need to drop)

In [ ]:
df.drop(columns=['redcap_repeat_instrument', 'tss_tegner'], inplace=True)

In [ ]:
df.info()

Looking for missing values in the rest of the columns

In [ ]:
df.isnull().sum() # looking for missing data

Finding the top few columns that have the least percentage of missing values, this shows that there exists a huge volume of missing values in the dataset and that imputation is very necessary in order to make any meaningful insight out of this dataset

In [ ]:
missing_percent = df.isnull().sum() / len(df) * 100
print(missing_percent.nsmallest(10))

# Seeing values to make the nan as Not reported

Before carrying out imputation for the missing values, replace all 'NaN' entries in the various demographic variables as Not reported for consistency

In [ ]:
print(df["sex_dashboard"].unique())
print(df['visit_sex'].unique())
print(df['age_group_dashboard_use'].unique())
print(df['graft_dashboard2'].unique())
print(df['visit_graft'].unique())

In [ ]:
print(df['redcap_event_name'].unique())

In [ ]:
print(df['strength_testing_complete'].unique())

In [ ]:
print(df['med_meniscus'].unique())
print(df['lat_meniscus'].unique())

1. Gender, Age Groups,  ( make the nan as Not reported) ( input)

In [ ]:
df['age_group_dashboard_use'] = df['age_group_dashboard_use'].fillna('Not Reported')
df['sex_dashboard'] = df['sex_dashboard'].fillna('Not Reported')
df['visit_sex'] = df['visit_sex'].fillna('Not Reported')

Visualizing and observing the presence of missing values across all columns and taking a look at what columns are useful, and what columns are meaningless

In [ ]:
# I've also included a visual aid of how many missing values we have for each variable
# Bar plot of missing values for each column
missing_vals = df.isnull().sum()
missing_vals = missing_vals[missing_vals > 0] # making sure to inlcude only the missing values ( no variables with complete)
missing_vals.sort_values(inplace=True)

plt.figure(figsize=(14, 10))
missing_vals.plot(kind='barh', color='lightcoral') # making this horizontal bargraph to enhance readability
plt.title('Missing Values Visualized', fontsize=20)# let's make sure the title is big
plt.xlabel('# of Missing Values ', fontsize=10)
plt.ylabel('Columns', fontsize=10)
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.gca().invert_yaxis() # we need this code to ensure that we have the missing values inverted
plt.show()

In [ ]:
# This leads us to see the distribution of the data and see outliers present ( use boxplot since they do a good job detecting/visualizing outliers)
plt.figure(figsize=(12, 12))
sns.boxplot(data=df, orient="h")
plt.title("Outlier present")
plt.show()

Exploratory phase: to impute data more effectively, we chose to split the imputation process by categorical and numerical variables. We chose to impute the categorical variables via the frequency of a cateogory's presence in the dataset, and impute the numerical variables via the mean of the column's numerical values.

In [ ]:
# Let's split the categorical and numerical columns
categorical_columns = df.select_dtypes(include=['object']).columns # this includes strings
numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns # these are numbers with meaning

# Now let's impute categorical(mode) and numerical columns (median)
df[categorical_columns] = df[categorical_columns].fillna(df[categorical_columns].mode().iloc[0])
df[numerical_columns] = df[numerical_columns].fillna(df[numerical_columns].median())

In [ ]:
df.isnull().sum()

Visualizing the distribution of graft types by age groups: are graft types a significant factor / choice among a certain age group?

In [ ]:
# distribtion of grafttype by age groups

plt.figure(figsize=(12, 6))
sns.countplot(data=df, x='graft_dashboard2', hue='age_group_dashboard_use')
plt.title("Distribution by Age Group")
plt.show()

Visualizing the distribution of age groups by sex category: this graph tells us that that the age group of 15-17 years old showcases a little disparity while all other age groups contain a similar number of male and female patients. An initial conclusion could be made that girls are likely to be injured from the age of 15 to 17 years old.

In [ ]:
# distrubtion of grafttype by sex
plt.figure(figsize=(12, 6))
sns.countplot(data=df, x='age_group_dashboard_use', hue='visit_sex')
plt.title("Distribution of Age by Sex")
plt.show()

We discovered that tss stands for 'time since surgery' a relatively useful variable for us since we are aiming to visualize the change in recovery rates or other variables over time.

In [ ]:
print(df['tss_dashboard'].unique()) # time since surgery

Visualizing how the self-reported pain evaluation score is distributed by the variable of time since surgery. If the KOOS pain score is distributed from 0 to 100, with 100 representing maximum 'pain level', we could conclude that most people felt maximum painfulness at the 8th to 12th month since surgery. This proves that that tss, combined with any other recovery metrics, could deduce valuable and interesting insights for not just us, but also our 'stakeholders' - the doctors/physicians when observing the recovery of their patients.

In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(data=df, x='koos_pain', hue='tss_dashboard', multiple='stack')  # or 'dodge'
plt.title("Distribution of KOOS Pain Scores by Time Since Surgery")
plt.xlabel("KOOS Pain Score")
plt.ylabel("Count")
plt.legend(df['tss_dashboard'])
plt.show()


With this in mind, we've decided to visualize tss against all of the other recovery metric variables (whether it is pain, LSI, strength score, etc) to see how patients recover over time. We adopted a small multiples strategy so that all of the visualizations can be created at once

In [ ]:
# visualizing the distirbution of strength testing, tss ( multiple)

fig,axs = plt.subplots(3,3, figsize=(20, 15))

sns.histplot(data=df, x='acl_sh', hue= 'tss_dashboard', multiple='stack', ax=axs[0,0], legend=False)
axs[0,0].set_title("ACL Strength")

sns.histplot(data=df, x='acl_th', hue= 'tss_dashboard', multiple='stack', ax=axs[0,1], legend=False)

sns.histplot(data=df, x='acl_ch', hue= 'tss_dashboard', multiple='stack', ax=axs[0,2], legend=False)

sns.histplot(data=df, x='acl_ext_mvic_90', hue= 'tss_dashboard', multiple='stack', ax=axs[1,0], legend=False)

sns.histplot(data=df, x='acl_ext_mvic_60', hue= 'tss_dashboard', multiple='stack', ax=axs[1,1], legend=False)

sns.histplot(data=df, x='acl_flex_mvic_60', hue= 'tss_dashboard', multiple='stack', ax=axs[1,2], legend=False)

sns.histplot(data=df, x='acl_ext_isok_60', hue= 'tss_dashboard', multiple='stack', ax=axs[2,0], legend=False)

sns.histplot(data=df, x='acl_flex_isok_60', hue= 'tss_dashboard', multiple='stack', ax=axs[2,1])

plt.show()

From the small multiples above, we could see how the recovery metrics are distributed by comparing the different 'types' of recovery metrics in the same 'category'. I.e. observing how acl knee extension value differentiates in the different degrees of which the patient can extend their knee to.